In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as f
# import torch.utils.data as data
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import numpy as np
import PIL 

import tensorflow as tf

In [2]:
import os
import sys

MODEL_PATH = '../models'
if MODEL_PATH not in sys.path:
    sys.path.append(MODEL_PATH)

in_dir='../dataset/color/'
img_paths = [x.path for x in os.scandir(in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
len(img_paths)

5578

In [3]:
loader = torchvision.transforms.Compose([
#     transforms.Scale(imsize), 
    torchvision.transforms.ToTensor()
])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = PIL.Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return image #assumes that you're using GPU

image = image_loader(img_paths[0])
image=torch.nn.functional.interpolate(image,(image.shape[2]//2, image.shape[3]//2)) # back to original resolution
image.shape

torch.Size([1, 3, 484, 648])

resnet的basicblock构造还有点问题，没有生成新的层

In [4]:
import resnet_detector
model = resnet_detector.DetectorModel()
optimizer = optim.SGD(model.parameters(),lr=0.001)

optimizer.zero_grad()
score_maps_list,endpoints = model(image)

0
1
2


# build_multi_scale_deep_detector_3DNMS

In [5]:
photos=image
batch_size = photos.shape[0]
height = photos.shape[2]
width = photos.shape[3]
C = photos.shape[1]

scale_factors = endpoints['scale_factors']
scale_factors_tensor=torch.tensor(scale_factors)
num_scale = len(score_maps_list)
scale_logits = [None] * num_scale

```
def instance_normalization(inputs):
    # normalize 0-means 1-variance in each sample (not take batch-axis)
    inputs_dim = inputs.get_shape().ndims
    # Epsilon to be used in the tf.nn.batch_normalization
    var_eps = 1e-3    
    if inputs_dim == 4:
        moments_dims = [1,2] # NHWC format
    elif inputs_dim == 2:
        moments_dims = [1]
    else:
        raise ValueError('instance_normalization suppose input dim is 4: inputs_dim={}\n'.format(inputs_dim))
    mean, variance = tf.nn.moments(inputs, axes=moments_dims, keep_dims=True)
    outputs = tf.nn.batch_normalization(inputs, mean, variance, None, None, var_eps) # non-parametric normalization
    return outputs
```
================torch.nn.BatchNorm2d(1, eps=1e-06, momentum=None, affine=None, track_running_stats=None)

In [6]:
instance_normalization=torch.nn.BatchNorm2d(1, eps=1e-06, momentum=None, affine=None, track_running_stats=None)
for i in range(num_scale):
    logits = instance_normalization(score_maps_list[i])
    logits = torch.nn.functional.interpolate(score_maps_list[i],(height, width)) # back to original resolution
    scale_logits[i] = logits
scale_logits=torch.cat(scale_logits, dim=1) # [B,S,H,W]

## soft_nms_3d(scale_logits, ksize, com_strength=1.0):
    # apply softmax on scalespace logits
    # scale_logits: [B,S,H,W]


In [8]:
def soft_nms_3d(scale_logits, ksize, com_strength=1.0):
    # apply softmax on scalespace logits
    # scale_logits: [B,H,W,S]
    num_scales =scale_logits.shape[1]
    # [B,S,H,W] -> (B,C,S,H,W)--batch size , channel 
    
    scale_logits_d=scale_logits[...,None].permute(0,4,1,2,3) # [B,S,H,W,1] in order to apply pool3d
    maxpool3d=torch.nn.MaxPool3d((num_scales, ksize, ksize),stride=(num_scales, 1, 1),padding=(0,(ksize-1)//2,(ksize-1)//2),dilation=1)
    max_maps=maxpool3d(scale_logits_d)
    max_maps = max_maps.squeeze(1) # (B,C,S,H,W) -> [B,S,H,W]
    exp_maps = torch.exp(com_strength * (scale_logits-max_maps))
    
    exp_maps_d=exp_maps[...,None].permute(0,4,1,2,3)
    conv3d=torch.nn.Conv3d(1, 1, (num_scales,ksize,ksize), stride=(num_scales, 1, 1), padding=(0,(ksize-1)//2,(ksize-1)//2), dilation=1,  bias=False)
    sum_ex=conv3d(exp_maps_d)
    sum_ex = sum_ex.squeeze(1)
    probs = exp_maps / (sum_ex + 1e-6)  

    return probs

In [9]:
scale_heatmaps = soft_nms_3d(scale_logits, ksize=15, com_strength=3)

In [10]:
scale_heatmaps.shape

torch.Size([1, 3, 484, 648])

## soft_max_and_argmax_1d
(inputs, axis=-1, inputs_index=None, keep_dims=False, com_strength1=250.0, com_strength2=250.0)

In [11]:
def soft_max_and_argmax_1d(inputs, axis=2, inputs_index=None, keep_dims=False, com_strength1=250.0, com_strength2=250.0):
    # Safe softmax
    inputs_max,_=torch.max(inputs, dim=axis, keepdim=keep_dims, out=None)
    inputs_exp1 = torch.exp(com_strength1*(inputs - inputs_max))
    inputs_softmax1_max,_=torch.max(inputs_exp1, dim=axis, keepdim=keep_dims, out=None)
    inputs_softmax1 = inputs_exp1 / (inputs_softmax1_max + 1e-8)

    inputs_exp2 = torch.exp(com_strength2*(inputs - inputs_max))
    inputs_softmax2_max,_=torch.max(inputs_exp2, dim=axis, keepdim=keep_dims, out=None)
    inputs_softmax2 = inputs_exp2 / (inputs_softmax2_max + 1e-8)

    inputs_max,_=torch.max(inputs * inputs_softmax1, dim=axis, keepdim=keep_dims, out=None)
    
    inputs_index_shp = [1,]*(len(inputs.shape)-1)
    inputs_index_shp[axis] = -1
    if inputs_index is None:
        inputs_index = torch.arange(inputs.shape[axis],dtype=inputs.dtype) # use 0,1,2,..,inputs.shape[axis]-1
    inputs_index = inputs_index.view(1,-1,1).squeeze().unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
    inputs_amax = torch.sum(inputs_index.float() * inputs_softmax2, dim=axis, keepdim=keep_dims)
    
    return inputs_max, inputs_amax,inputs_softmax2

In [12]:
max_heatmaps, max_scales,inputs_softmax2 = soft_max_and_argmax_1d(scale_heatmaps, axis=1, 
                                    inputs_index=scale_factors_tensor, keep_dims=False,
                                    com_strength1=100,
                                    com_strength2=100) # both output = [B,H,W]

In [13]:
max_heatmaps.shape

torch.Size([1, 484, 648])

In [14]:
max_heatmaps = max_heatmaps[..., None].permute(0,3,1,2)
max_heatmaps.shape

torch.Size([1, 1, 484, 648])

In [15]:
def end_of_frame_masks(height, width, radius, dtype=torch.float32):
    eof_masks = torch.ones([1,1,height-2*radius,width-2*radius], dtype=torch.float32)
    m = nn.ConstantPad2d(radius, 0)
    eof_masks = m(eof_masks)
    return eof_masks

In [16]:
endpoints['pad_size'] = 8 * (3//2)
endpoints['pad_size']

8

现在要做的是把边缘裁剪掉，可能有两个原因：
    一是边缘不容易生成patch
    二是前面padding操作对边缘可能不好影响

In [17]:
eof_masks_pad = end_of_frame_masks(height, width, endpoints['pad_size'])
eof_masks_pad.shape

torch.Size([1, 1, 484, 648])

In [18]:
max_heatmaps = max_heatmaps * eof_masks_pad

## Extract Top-K keypoints

In [19]:
eof_masks_crop = end_of_frame_masks(height, width, 16)
eof_masks_crop.shape

torch.Size([1, 1, 484, 648])

### non_max_suppression  
函数主要思想感觉是非极大抑制，首先在input上padding出来一圈0（kernal大小），然后在左上角抖动产生ksize×ksize个子窗口，只有kernalsize大小的极大值才能被选中

In [20]:
def non_max_suppression(inputs, thresh=0.0, ksize=3, dtype=torch.float32):
    dtype = inputs.dtype
    batch = inputs.shape[0]
    height = inputs.shape[2]
    width = inputs.shape[3]
    channel = inputs.shape[1]
    hk = ksize // 2
    zeros = torch.zeros_like(inputs)
    works = torch.where(inputs<thresh,zeros,inputs)
    m = nn.ConstantPad2d(2*hk, 0)
    works_pad = m(works)
    map_augs = []
    for i in range(ksize):
        for j in range(ksize):
            curr_in = works_pad[:,:,i:i+height+2*hk,j:j+width+2*hk]
            map_augs.append(curr_in)

    num_map = len(map_augs) # ksize*ksize
    center_map = map_augs[num_map//2]
    peak_mask = center_map > map_augs[0]
    for n in range(1, num_map):
        if n == num_map // 2:
            continue
        peak_mask = peak_mask & (center_map > map_augs[n])
    peak_mask = peak_mask[:,:,hk:height+hk,hk:width+hk]
    
#     peak_mask.set_shape(inputs.shape) # keep shape information
    return peak_mask


In [21]:
nms_maps = non_max_suppression(max_heatmaps, 0, 5)
# print(max_heatmaps.dtype)
# print(nms_maps.dtype)
# print(eof_masks_crop.dtype)

nms_scores = max_heatmaps * nms_maps.float() * eof_masks_crop
# nms_scores[0][0][200]

In [22]:
def make_top_k_sparse_tensor(heatmaps, k=256, get_kpts=False):
    batch_size = heatmaps.shape[0]
    height = heatmaps.shape[2]
    width = heatmaps.shape[3]
    channel = heatmaps.shape[1]
    
    def sparse_to_dense(indices, out_shape):
        O = torch.zeros(out_shape)
        for i, p in enumerate(indices):
            O[p] = 1
        return O
    
    heatmaps_flt = heatmaps.reshape([batch_size, -1])
    imsize = heatmaps_flt.shape[1]

    values, xy_indices=torch.topk(heatmaps_flt, k=k, dim=1, largest=True, sorted=False)     
    boffset = (torch.arange(0,batch_size) * imsize).unsqueeze(1)
    indices = xy_indices + boffset
    indices = indices.reshape([-1])
    top_k_maps = sparse_to_dense(indices,[batch_size*imsize])
    top_k_maps = top_k_maps.reshape([batch_size, 1, height, width])
    top_k_maps.float()
    if get_kpts:
#         kpx = tf.mod(xy_indices, width)
#         kpy = xy_indices // width
#         batch_inds = tf.tile(tf.range(batch_size, dtype=tf.int32)[:,None], [1,k])
#         kpts = tf.concat([tf.reshape(kpx, [-1,1]), tf.reshape(kpy, [-1,1])], axis=1) # B*K,2
#         batch_inds = tf.reshape(batch_inds, [-1])
#         num_kpts = tf.ones([batch_size], dtype=tf.int32) * k
#         return top_k_maps, kpts, batch_inds, num_kpts
        pass
    else:
        return top_k_maps 

In [331]:
top_ks = make_top_k_sparse_tensor(nms_scores, k=256)
# top_ks.dtype
top_ks = top_ks * nms_maps.float() # 这一步感觉没任何意义
top_ks = top_ks.requires_grad_(False)

ori_maps = endpoints['ori_maps']

In [302]:
def extract_keypoints(top_k):
    coords=[]
#     batch_inds=top_k.shape[0]
    batch_inds,_,_,_=np.where(top_ks.numpy()>0)
    num_kpts=[]
    for batch in range(top_k.shape[0]):
        _,x,y=np.where(top_ks[batch].numpy()>0)
        for i,j in zip(x,y):
            coords.append([i,j])
        num_kpts.append(len(x))
    coords=np.array(coords)
    num_kpts = torch.tensor(num_kpts)
    kpts=torch.from_numpy(coords)
#     batch_inds=len(batch_inds)
    # kpts: [N,2] (N=B*K)
    # batch_inds: N,
    # num_kpts: B
    return kpts, batch_inds, num_kpts

In [303]:
kpts, batch_inds, num_kpts = extract_keypoints(top_ks)

In [248]:
kpts.shape

torch.Size([256, 2])

In [306]:
batch_inds[:None].shape

(256,)

In [257]:
num_kpts

tensor([256])

In [361]:
def batch_gather_keypoints(inputs, batch_inds, kpts, xy_order=True):
    # kpts: [N,2] x,y or y,x
    # batch_inds: [N]
    # outputs = inputs[b,y,x]
    output=[]
    if xy_order:
        kp_x, kp_y = np.array_split(kpts.numpy(), 2,axis=1)
    else:
        kp_y, kp_x = np.array_split(kpts.numpy(), 2,axis=1)
    batch_inds = batch_inds[:,None]
    byx = torch.cat([torch.from_numpy(batch_inds), torch.from_numpy(kp_x), torch.from_numpy(kp_y)], dim=1)
    for i,j,k in byx:
        temp=inputs[int(i)][int(j)][int(k)]
        output.append(temp.data)
    return output


In [362]:
kpts_scale = batch_gather_keypoints(max_scales, batch_inds, kpts)
kpts_scale

[tensor(1.),
 tensor(1.),
 tensor(0.3536),
 tensor(0.1250),
 tensor(0.3536),
 tensor(1.),
 tensor(0.3536),
 tensor(0.3536),
 tensor(1.),
 tensor(1.),
 tensor(0.3536),
 tensor(0.3536),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(0.1250),
 tensor(0.1250),
 tensor(0.1250),
 tensor(1.),
 tensor(1.),
 tensor(0.1250),
 tensor(0.3536),
 tensor(0.3536),
 tensor(0.1250),
 tensor(0.1250),
 tensor(0.3536),
 tensor(0.1250),
 tensor(1.),
 tensor(0.3536),
 tensor(0.3536),
 tensor(0.3536),
 tensor(0.3536),
 tensor(0.1250),
 tensor(1.),
 tensor(0.1250),
 tensor(0.1250),
 tensor(1.),
 tensor(0.1250),
 tensor(1.),
 tensor(0.1250),
 tensor(0.3536),
 tensor(0.1250),
 tensor(0.1250),
 tensor(0.3536),
 tensor(0.1250),
 tensor(0.1250),
 tensor(0.1250),
 tensor(1.),
 tensor(0.1250),
 tensor(0.1250),
 tensor(1.),
 tensor(0.3536),
 tensor(0.3536),
 tensor(0.1250),
 tensor(0.1250),
 tensor(1.),
 tensor(0.1250),
 tensor(1.),
 tensor(0.1250),
 tensor(0.3536),
 tensor(1.),
 tensor(1.),
 tensor(0.1250),
 tensor(0

In [316]:
import gc
gc.collect()

8939